In [2]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
import pdb
import fxns
reload(fxns);

In [4]:
raw_data_path = '../data/shuffled_examples'

outbase = '../output/shuffled_examples'
train_out_path = '%s/texttrain' % outbase
valid_out_path = '%s/textvalidate' % outbase
test_out_path = '%s/texttest' % outbase

test_chrs = [22]
valid_chrs = [21]

blocksize = 10 # the number of records in each generated file

num_blocks = 2 # the number of blocks to write out. small for debugging

text = False
binary = True

seq_len = 1000 # length in base-pairs of training sequences

In [5]:
# split data into train/validate/test
fxns.create_shards(raw_data_path,
            train_out_path, valid_out_path, test_out_path,
            test_chrs, valid_chrs,
            blocksize=blocksize,
            text=text, binary=binary, num_blocks=num_blocks)

OSError: [Errno 2] No such file or directory: '../data/shuffled_examples'

In [ ]:
# print some data to make sure we got it right
# get training data
tf.reset_default_graph()
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label],
    batch_size=10,
    capacity=50,
    min_after_dequeue=50)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # read one batch of seqs and labels
    _seq_batch, _label_batch = sess.run([seq_batch, label_batch])
    
    # print their shapes
    print _seq_batch.shape, _label_batch.shape
    
    # see if we can re-shape the sequences to what we need
    print sess.run(tf.reshape(seq_batch, (-1, seq_len, 4))).sum(axis=0)

    coord.request_stop()
    coord.join(threads)